In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import datetime
import random
import re
import pprint
import json
import pandas as pd

def getMatchLinks (articleUrl):
    html = urlopen("http://www.espn.co.uk/rugby/fixtures/_/date/" + articleUrl)
    bsObj = bs(html, "lxml")
    tst = bsObj.find("section", {"id":"pane-main"}).findAll("script", {"type":"text/javascript"})
    for t in tst:
        json_text = re.search(r'^\s*window\.__INITIAL_STATE__\s*=\s*({.*?})\s*;\s*$',
                      t.string, flags=re.DOTALL | re.MULTILINE).group(1)

    json_out = json.loads(json_text)
    
    # Get just the match links
    json_match = json_out['schedule']['groups']
    matches = []
    for m in json_match:
        for c in m['complete']:
            matches.append(c['result']['href'])
            
    return matches

links = getMatchLinks('20160917')
links

['/rugby/report?gameId=289315&league=244293',
 '/rugby/report?gameId=268579&league=244293',
 '/rugby/report?gameId=290006&league=267979',
 '/rugby/report?gameId=290007&league=267979',
 '/rugby/report?gameId=290008&league=267979',
 '/rugby/match?gameId=290268&league=270559',
 '/rugby/match?gameId=290265&league=270559',
 '/rugby/match?gameId=290266&league=270559',
 '/rugby/match?gameId=290270&league=270559',
 '/rugby/match?gameId=290269&league=270559',
 '/rugby/report?gameId=290441&league=270557',
 '/rugby/report?gameId=290440&league=270557',
 '/rugby/match?gameId=290152&league=270555',
 '/rugby/match?gameId=290149&league=270555',
 '/rugby/match?gameId=289875&league=289232',
 '/rugby/match?gameId=290197&league=270563',
 '/rugby/match?gameId=290196&league=270563']

In [2]:
def getStatsLinks (articleUrl):
    html = urlopen("http://www.espn.co.uk" + articleUrl)
    bsObj = bs(html, "lxml")
    tst = bsObj.find("section", {"id":"pane-main"}).findAll("script", {"type":"text/javascript"})
    
    json_out = []
    for t in tst:
        json_text = re.search(r'^\s*window\.__INITIAL_STATE__\s*=\s*({.*?})\s*;\s*$',
                          t.string, flags=re.DOTALL | re.MULTILINE).group(1)
        json_out.append(json.loads(json_text))

    for j in json_out[0]['gamePackage']['links']:
        if j['pageType'] == 'matchstats':
            match_s = j['href']
        elif j['pageType'] == 'playerstats':
            player_s = j['href']
          
    try: 
        match_s, player_s
    except NameError:
        match_s, player_s = None, None

    return match_s, player_s

m, p = getStatsLinks(links[5])
print(m)
print(p)

/rugby/matchstats?gameId=290268&league=270559
/rugby/playerstats?gameId=290268&league=270559


In [3]:
def getStats(statsUrl):
    html = urlopen("http://www.espn.co.uk" + statsUrl)
    bsObj = bs(html, "lxml")  
    tst = bsObj.find("section", {"id":"pane-main"}).findAll("script", {"type":"text/javascript"})
    
    stat_out = []
    for t in tst:
        json_text = re.search(r'^\s*window\.__INITIAL_STATE__\s*=\s*({.*?})\s*;\s*$',
                              t.string, flags=re.DOTALL | re.MULTILINE).group(1)
        stat_out.append(json.loads(json_text))
        
    return stat_out[0]

In [4]:
def getPlayerVals(d):

    dct = d
    dct['match_date'] = date
    dct['match_id'] = match_id[0] # regex above returns list of one element
    if 'id' in dct:
        dct['player_id'] = dct.pop('id') # rename 'id' to 'player_id'
    # This value should be unique and will be the primary key of the table
    dct['row_nk'] = dct['player_id'] + dct['match_id'] 

    # remove entries we don't want - url is useless and eventTimes doesn't conform to the required structure
    # these are the keys, i.e. table column names
    cols = list(dct.keys())
    cols = list(filter(lambda c: c != 'eventTimes', cols))
    cols = list(filter(lambda c: c != 'url', cols))

    # these are used for the SQL query, so everything gets inserted at once
    placeholders = ', '.join(['%s'] * len(cols))
    columns = ', '.join(cols)

    # These are the values to be inserted into the table. Sometimes the values is a single value
    # and other times is a dictionary, handle both instances
    vals = []
    for d in dct:
        if d in cols:
            if type(dct[d]) is dict:
                vals.append(dct[d]['value'])
            else:
                vals.append(dct[d])

    return placeholders, columns, vals

In [70]:
def getMatchVals(m_stats):
    # home team name from player stats
    home = m_stats['gamePackage']['gameStrip']['teams']['home']['name']
    # Away team name from player stats:
    away = m_stats['gamePackage']['gameStrip']['teams']['away']['name']
    # Stadium from player stats:
    ground = m_stats['gamePackage']['matchDetails']['venue']
    # Match date from player stats:
    dt = m_stats['gamePackage']['matchDetails']['date']
    # Competition from player stats:
    comp = m_stats['gamePackage']['gameStrip']['header']
    # Get the date as a date object, from match stats:
    try:
        as_dt = datetime.strptime(dt, '%I:%M PM, %B %d, %Y')
    except ValueError:
        as_dt = datetime.strptime(dt, '%I:%M AM, %B %d, %Y')
    # Exract the date only poriton
    dt_only = as_dt.date()
    # Extract the time portion
    tm = as_dt.time()
    # Year only
    yr = as_dt.year
    # Month only
    mth = as_dt.month
    # Day only
    dy = as_dt.day
    
    return [home, away, ground, dt, comp, as_dt, dt_only, tm, yr, mth, dy]

In [26]:
# query to create the table in MySQL - this will need a match and player identifier
sql = """drop table if exists scraping.p_stats;
        create table p_stats( row_nk varchar(50) not null
                            , player_id bigint(7) not null
                            , match_id varchar(20) not null
                            , match_date varchar(20)
                            , name varchar(200)
                            , number varchar(10)
                            , position varchar(10)
                            , captain varchar(10)
                            , subbed varchar(10)
                            , homeAway varchar(10)
                            , subToolTip varchar(10)
                            , onPitch varchar(10)
                            , wasActive varchar(10)
                            , tries bigint(7)
                            , tryassists bigint(7)
                            , points bigint(7)
                            , kicks bigint(7)
                            , passes bigint(7)
                            , runs bigint(7)
                            , metres bigint(7)
                            , cleanbreaks bigint(7)
                            , defendersbeaten bigint(7)
                            , offload bigint(7)
                            , lineoutwonsteal bigint(7)
                            , turnoversconceded bigint(7)
                            , tackles bigint(7)
                            , missedtackles bigint(7)
                            , lineoutswon bigint(7)
                            , penaltiesconceded bigint(7)
                            , yellowcards bigint(7)
                            , redcards bigint(7)
                            , penalties bigint(7)
                            , penaltygoals bigint(7)
                            , conversiongoals bigint(7)
                            , dropgoalsconverted bigint(7)
                            , primary key(row_nk));""".replace("\n", "")

# Run this first to set up connection drop the table, and create again before putting more data into it

# Set up the connection
import pymysql
conn = pymysql.connect(host = '127.0.0.1', port = 3306,
                       user = 'root', passwd = '', db = 'mysql')

cur = conn.cursor()
cur.execute("USE scraping")
cur.execute(sql)

0

In [118]:
# query to create the match details table
msql = """drop table if exists scraping.m_details;
        create table m_details( row_nk varchar(50) not null
                                , home_team varchar (50)
                                , away_team varchar (50)
                                , ground varchar (50)
                                , match_dttime_str varchar(50)
                                , competition varchar(50)
                                , match_dttime varchar(50)
                                , date varchar (20)
                                , time varchar (20)
                                , year varchar (10)
                                , month varchar (5)
                                , day varchar (5) );""".replace("\n", "")

# Run this first to set up connection drop the table, and create again before putting more data into it

# Set up the connection
import pymysql
conn = pymysql.connect(host = '127.0.0.1', port = 3306,
                       user = 'root', passwd = '', db = 'mysql')

cur = conn.cursor()
cur.execute("USE scraping")
cur.execute(msql)

0

In [115]:
dt = pd.date_range('20160801','20170319')
#dt = pd.date_range('20160801','20160831')
dates = dt.format(formatter=lambda x: x.strftime('%Y%m%d'))
#dates

In [87]:
# populate the player stats database. This fails sometimes for reasons I can't understand
# if it does, need to restart the Apache DB and probably drop the table, and try again
teams, players = ['home', 'away'], ['team', 'reserves']
# dates = ['20170304', '20170305'] # Testing with two dates
# dates = ['20160917']

for date in dates:
    print(date)
    matches = getMatchLinks(date)
    
    if len(matches) > 0:
        for match_link in matches:
            if match_link != '#':
                ma, pl = getStatsLinks(match_link)

                regex = re.compile('gameId=([0-9]*)')
                match_id = regex.findall(match_link)

                if pl != None:
                    p_stats = getStats(pl)

                    for t in teams:
                        for p in players:
                            data = p_stats["gamePackage"]["matchLineUp"][t][p]
                            for d in data:               
                                placeholders, columns, vals = getPlayerVals(d)

                                cur = conn.cursor()
                                cur.execute("USE scraping")

                                # Execute the SQL to update the table
                                sql_update = "insert into p_stats (%s) values (%s)" % (columns, placeholders)
                                cur.execute(sql_update, vals)
                                conn.commit()

cur.close()
conn.close()

20170304


IntegrityError: (1062, "Duplicate entry '82104290088' for key 'PRIMARY'")

In [119]:
# Populate the match details
for date in dates:
    print(date)
    matches = getMatchLinks(date)
    
    if len(matches) > 0:
        for match_link in matches:
            if match_link != '#':
                ma, pl = getStatsLinks(match_link)

                regex = re.compile('gameId=([0-9]*)')
                match_id = regex.findall(match_link)

                if ma != None:
                    m_stats = getMatchVals(getStats(ma))
                    m_stats.insert(0, match_id[0])

                    #columns = ['row_nk', 'home_team', 'away_team', 'ground','match_dttime_str', 'competition', 
                    #    'match_dttime', 'date', 'time', 'year', 'month', 'day']
                    cols = 'row_nk, home_team, away_team, ground, match_dttime_str, competition, match_dttime, date, time, year, month, day'
                    placeholders = ', '.join(['%s'] * len(columns))
                                    
                    cur = conn.cursor()
                    cur.execute("USE scraping;")

                    # Execute the SQL to update the table
                    sql_update = "insert into m_details (%s) values (%s);" % (cols, placeholders)
                    cur.execute(sql_update, m_stats)
                    conn.commit()

cur.close()
conn.close()


20160801
20160802
20160803
20160804
20160805
20160806
20160807
20160808
20160809
20160810
20160811
20160812
20160813
20160814
20160815
20160816
20160817
20160818
20160819
20160820
20160821
20160822
20160823
20160824
20160825
20160826
20160827
20160828
20160829
20160830
20160831
20160901
20160902
20160903
20160904
20160905
20160906
20160907
20160908
20160909
20160910
20160911
20160912
20160913
20160914
20160915
20160916
20160917
20160918
20160919
20160920
20160921
20160922
20160923
20160924
20160925
20160926
20160927
20160928
20160929
20160930
20161001
20161002
20161003
20161004
20161005
20161006
20161007
20161008
20161009
20161010
20161011
20161012
20161013
20161014
20161015
20161016
20161017
20161018
20161019
20161020
20161021
20161022
20161023
20161024
20161025
20161026
20161027
20161028
20161029
20161030
20161031
20161101
20161102
20161103
20161104
20161105
20161106
20161107
20161108
20161109
20161110
20161111
20161112
20161113
20161114
20161115
20161116
20161117
20161118
20161119
2

In [117]:
# Run this if the code fails to close the DB connection
cur.close()
conn.close()

Error: Already closed